# Load data

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
#!pip install PyDrive
#!pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Import partner1 data
import pandas as pd
downloaded = drive.CreateFile({'id':"1i03d8w4qnex4GlyXj12caaosHCGrpmej"})   # replace the id with id of file you want to access
downloaded.GetContentFile('partner1.xlsx') 
partner1 = pd.read_excel('partner1.xlsx')

In [0]:
# Import partner2 data
import pandas as pd
downloaded = drive.CreateFile({'id':"1EWAO7c700XTshY7MhMt-BcsUWlACjKu4"})   # replace the id with id of file you want to access
downloaded.GetContentFile('partner2.xlsx') 
partner2 = pd.read_excel('partner2.xlsx')

In [0]:
# Import example data
import pandas as pd
downloaded = drive.CreateFile({'id':"1JaZukEjWt_EqDVjvcmFOmdz__opIMOlw"})   # replace the id with id of file you want to access
downloaded.GetContentFile('example.xlsx') 
example = pd.read_excel('example.xlsx')

In [0]:
# Change column names of partner1 and partner2 dataframes
partner1.columns = ['key', 'hotel_name', 'city_name', 'country_code', 'hotel_address', 'star_rating', 'postal_code']
partner2.columns = ['key', 'hotel_name', 'city_name', 'country_code', 'hotel_address', 'star_rating', 'postal_code']

In [0]:
# Split into example1 and example2, standarize column names
example1 = example[['p1.key', 'p1.hotel_name', 'p1.city_name', 'p1.country_code', 'p1.hotel_address', 'p1.star_rating', 'p1.postal_code']]
example1.columns = ['key', 'hotel_name', 'city_name', 'country_code', 'hotel_address', 'star_rating', 'postal_code']
example2 = example[['p2.key', 'p2.hotel_name', 'p2.city_name', 'p2.country_code', 'p2.hotel_address', 'p2.star_rating', 'p2.postal_code']]
example2.columns = ['key', 'hotel_name', 'city_name', 'country_code', 'hotel_address', 'star_rating', 'postal_code']

# Data Preprocessing and Filtering

## Unifiy the datasets adn initialize final dataframe

In [0]:
unified1 = partner1.append(example1)
unified2 = partner2.append(example2)
matched = pd.DataFrame(columns = ['key', 'predicted_key'])

## Quick look

We see that our data is mostly full. Hotel name has no missing values on any of the datasets. We see that partner1 has more duplicated hotel names than partner2. 

In [0]:
unified1.describe(include='all')

In [0]:
unified2.describe(include='all')

## Duplicate keys?

In [0]:
# Unified1
unified1[unified1.duplicated(subset=['key'], keep=False)]

In [0]:
# Unified2
unified2[unified2.duplicated(subset=['key'], keep=False)]

## Duplicate name and hotel?

Let's see if we have duplicates (we define duplicates as same hotel name, city code and country). If we do and they are not so many let's manually match them and take them out of the game.

In [0]:
# Unified1. We have a perfect duplicate, let's get rid of it.
unified1[unified1.duplicated(subset=['hotel_name', 'city_name', 'country_code'], keep=False)]

In [0]:
# Unified2. We see 2 hotels with the same name, city and country but they seem to be 2 different hotels.
unified2[unified2.duplicated(subset=['hotel_name', 'city_name', 'country_code'], keep=False)]

In [0]:
# Let's check if we can find hotels with that name un Unified1
unified1[unified1['hotel_name']=='Comfort Inn Lincoln']

In [0]:
# We have our first match. Let's take it out of the unified1 and unified2 dataframes
# and save it to the matched dataframe
unified1 = unified1.drop(unified1.index[[508,664]])
unified1 = unified1.reset_index(drop=True)
unified2 = unified2.drop(unified2.index[8540])
unified2 = unified2.reset_index(drop=True)

d = {'key': ['EF00463C2A4279D9D58D452CB713906B'], 'predicted_key': ['C22A4FFCF0BB6E0CAB25E7AE4501E4BD']}
row = pd.DataFrame(data=d)
matched = matched.append(row)
matched.head()

## Hotel Country

We see that few countries concentrate most of the data. They are US, TH, CN, IN, ID, JP, AU, VN, GB and more. Some intuition tells us that we may probably need to focus on those mostly.

In [0]:
unified1_grouped = unified1.groupby(['country_code']).count().reset_index()
unified1_grouped.sort_values('key', ascending=False, inplace=True)
plt.figure(figsize = (30,7))
sns.barplot(x='country_code',y='key', data = unified1_grouped, edgecolor=".1").set_title('Number of datapoints by country in Unified1')

In [0]:
unified2_grouped = unified2.groupby(['country_code']).count().reset_index()
unified2_grouped.sort_values('key', ascending=False, inplace=True)
plt.figure(figsize = (30,7))
sns.barplot(x='country_code',y='key', data = unified2_grouped, edgecolor=".1").set_title('Number of datapoints by country in Unified2')

There are some datapoints in both partners that have no country code. Let's have a look at them, match them if we can and delete them from our dataset.

In [0]:
unified1[unified1['country_code'].isnull()]

In [0]:
unified2[unified2['country_code'].isnull()]

In [0]:
# They all have a matching pair, let's add them to out matched dataframe
unified1 = unified1.drop(unified1.index[[4782,5733,6064,9774,9894]])
unified2 = unified2.drop(unified2.index[[1303,3134,5699,7776,9671]])

d = {'key': ['0AC418BC2D45A6B8518096F1F00AF00F', 'C8A0A5B634A67365D57AB0983E601C62',
             '4B36DF6237887FEE1B7A51FFF8F5F79F', 'CBE71D01B60AF5E074935BCE2F434AF4', 
             'F5782D38E38BE92661FC2A38A65335F1'], 
     'predicted_key': ['437A055FE6E759CCA7269E9F3AFAAA1B', '05E782BCA41AB39ED4F6602C9E54BAA8', 
                       'AEA31679E9873B89508E6A95DF23BBD8', '5256D45F46338BFFA68B84A7BEC99146', 
                       '9590C8BC8A30858FE272F3A08AD04173']}

row = pd.DataFrame(data=d)
matched = matched.append(row)

In [0]:
matched.reset_index(drop=True, inplace=True)
matched.head()

## Hotel postal_code

In [0]:
unified1['postal_code'].describe(include='all')

In [0]:
unified2['postal_code'].describe(include='all')

## Hotel City

In [0]:
unified1['city_name'].describe(include='all')

In [0]:
unified2['city_name'].describe(include='all')

Let's also have a quick look at how the city codes look like in the main countries. Let's begin with the US, where we see that partner1 uses the (XX) state code while partner2 doesn't. This may be useful later.

### USA

In [0]:
unified1[unified1['country_code']=='US']['city_name'].head(10)

In [0]:
unified2[unified2['country_code']=='US']['city_name'].head(10)

### China

In [0]:
unified1[unified1['country_code']=='CN']['city_name'].head(20)

In [0]:
unified2[unified2['country_code']=='CN']['city_name'].head(20)

### Thailand

In [0]:
unified1[unified1['country_code']=='TH']['city_name'].head(20)

In [0]:
unified2[unified2['country_code']=='TH']['city_name'].head(20)

## Hotel Address

We see that on unified1 there are 22 hotels with address ".", and on unified2 there are 21 hotels with address " ". We will change those for NaNs

In [0]:
unified1['hotel_address'].describe(include='all')

In [0]:
unified2['hotel_address'].describe(include='all')

In [0]:
# Clean addresses
unified1['hotel_address'] = unified1.apply(lambda x: '' if x['hotel_address']=='.' else x['hotel_address'], axis=1)
unified2['hotel_address'] = unified2.apply(lambda x: '' if x['hotel_address']==' ' else x['hotel_address'], axis=1)

## Star rating

Looks legit. It seems that on average partner1 likes to give more starts than partner2.

In [0]:
unified1['star_rating'].describe(include='all')

In [0]:
unified2['star_rating'].describe(include='all')

## Hotel Name

It would be interesting to see which are the most frequent words. Probably "Hotel" or "Inn", but is it the same frequency  on both partners? If one of the partners uses the word "Hotel" everytime and the other one doesn't it may confuse our string matching model.  

Indeed we see that the word "Hotel" is the most popular one, but its there on 4368 datapoints on partner 1 while in only 3502 datapoints on partner 2. Maybe we should just simply delete these most common words from the hotel names, for example just "Hotel" and "Inn" may improve the results. We'll think about that later.

In [0]:
unified1['hotel_name'].str.split(expand=True).stack().value_counts().head(15)

In [0]:
unified2['hotel_name'].str.split(expand=True).stack().value_counts().head(15)

# Perfect Matches: The easy part

If there is a listing with the exact same hotel name, city and country then they are a match. We won't be lucky enough to find that most of our dataset is like that but we can at least split those and reduce the size.

We have 3045 out of 10500 (~30%) that are a perfect match. We can take them out from the unified dataset. Note that if would have done the same without caring at the city name we could have splitted about 5300 hotels, meaning there are about 2500 hotels with same name and same country but different city.

In [0]:
temp1 = unified1.copy()
temp2 = unified2.copy()
temp1['hotel_name2'] = temp1['hotel_name'].str.lower()
temp1['city_name2'] = temp1['city_name'].str.lower()
temp1['country_code2'] = temp1['country_code'].str.lower()
temp2['hotel_name2'] = temp2['hotel_name'].str.lower()
temp2['city_name2'] = temp2['city_name'].str.lower()
temp2['country_code2'] = temp2['country_code'].str.lower()

In [0]:
obvious = pd.merge(temp1, temp2, how='inner', on=['hotel_name2', 'city_name2', 'country_code2'])
obvious = obvious[['key_x', 'key_y']]
obvious.columns = ['key', 'predicted_key']
obvious.shape[0]

In [0]:
# So this is what we have so far and that we are greatly confident about
# 3045 out of 10498 --> 30%
matched = matched.append(obvious)
matched = matched.reset_index(drop=True)
matched.shape[0]

In [0]:
# Drop the perfectly matched hotels from each unified1 and unified2
unified1_remain = unified1.merge(matched, left_on=['key'], right_on=['key'], how='left', indicator=True)
unified1_remain = unified1_remain[unified1_remain['_merge']=='left_only']
unified1_remain.drop(columns=['_merge', 'predicted_key'], inplace=True)
print(unified1_remain.shape[0])

unified2_remain = unified2.merge(matched, left_on=['key'], right_on=['predicted_key'], how='left', indicator=True)
unified2_remain = unified2_remain[unified2_remain['_merge']=='left_only']
unified2_remain.drop(columns=['_merge', 'key_y', 'predicted_key'], inplace=True)
unified2_remain = unified2_remain.rename(columns = {'key_x':'key'})
print(unified2_remain.shape[0])

In [0]:
# Get datapoints inside example that were not yet matched
example_remain = example.merge(matched, left_on=['p1.key'], right_on=['key'], how='left', indicator=True)
example_remain = example_remain[example_remain['_merge']=='left_only']
example_remain.drop(columns=['key', 'predicted_key', '_merge'], inplace=True)
example_remain.shape[0]

In [0]:
example_remain = example_remain.merge(matched, left_on=['p2.key'], right_on=['predicted_key'], how='left', indicator=True)
example_remain = example_remain[example_remain['_merge']=='left_only']
example_remain.drop(columns=['key', 'predicted_key', '_merge'], inplace=True)
example_remain.shape[0]

In [0]:
#SAVEEE
from google.colab import files
caca = matched.to_csv("matched_after_obvious.csv", index=False)
files.download('matched_after_obvious.csv')
caca = unified1_remain.to_csv("unified1_remain_after_obvious.csv", index=False)
files.download('unified1_remain_after_obvious.csv')
caca = unified2_remain.to_csv("unified2_remain_after_obvious.csv", index=False)
files.download('unified2_remain_after_obvious.csv')
caca = example_remain.to_csv("example_remain_after_obvious.csv", index=False)
files.download('example_remain_after_obvious.csv')

# Measuring Similarity

Let's think now on how we can do fuzzy string matching since we already took out the perfectly matching hotels. We can define a scoring function and see how it behaves on our labeled data: the examples dataset. We will apply it only on the datapoints that are not a perfect match, otherwise we will see very high scores for perfectly matching names that will corrupt our intuition.  

From the total 499 datapoints on example dataset 378 of them are not a perfect match

In [0]:
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz

Let's use our 4 fuzzy matching algorithms:
- Ratio
- Partial Ratio
- Token Ratio
- Token Set Ratio

## When there is a match

### Hotel Name

In [0]:
example_remain['name_ratio'] = example_remain[['p1.hotel_name', 'p2.hotel_name']].apply(lambda x: fuzz.ratio(x['p1.hotel_name'], x['p2.hotel_name']), axis=1)
example_remain['name_partial_ratio'] = example_remain[['p1.hotel_name', 'p2.hotel_name']].apply(lambda x: fuzz.partial_ratio(x['p1.hotel_name'], x['p2.hotel_name']), axis=1)
example_remain['name_token_sort_ratio'] = example_remain[['p1.hotel_name', 'p2.hotel_name']].apply(lambda x: fuzz.token_sort_ratio(x['p1.hotel_name'], x['p2.hotel_name']), axis=1)
example_remain['name_token_set_ratio'] = example_remain[['p1.hotel_name', 'p2.hotel_name']].apply(lambda x: fuzz.token_set_ratio(x['p1.hotel_name'], x['p2.hotel_name']), axis=1)

In [0]:
plt.figure(figsize = (25,14))
plt.subplot(2,2,1)
plt.title('Hotel Name: Using the Ratio' ,color = "g")
sns.distplot(example_remain['name_ratio'], kde=False);
plt.subplot(2,2,2)
plt.title('Hotel Name: Using the Partial Ratio' ,color = "g")
sns.distplot(example_remain['name_partial_ratio'], kde=False);
plt.subplot(2,2,3)
plt.title('Hotel Name: Using the Token Sort Ratio' ,color = "g")
sns.distplot(example_remain['name_token_sort_ratio'], kde=False);
plt.subplot(2,2,4)
plt.title('Hotel Name: Using the Token Set Ratio' ,color = "g")
sns.distplot(example_remain['name_token_set_ratio'], kde=False);

### Hotel Address

In [0]:
example_remain['address_ratio'] = example_remain[['p1.hotel_address', 'p2.hotel_address']].apply(lambda x: fuzz.ratio(str(x['p1.hotel_address']), str(x['p2.hotel_address'])), axis=1)
example_remain['address_partial_ratio'] = example_remain[['p1.hotel_address', 'p2.hotel_address']].apply(lambda x: fuzz.partial_ratio(str(x['p1.hotel_address']), str(x['p2.hotel_address'])), axis=1)
example_remain['address_token_sort_ratio'] = example_remain[['p1.hotel_address', 'p2.hotel_address']].apply(lambda x: fuzz.token_sort_ratio(str(x['p1.hotel_address']), str(x['p2.hotel_address'])), axis=1)
example_remain['address_token_set_ratio'] = example_remain[['p1.hotel_address', 'p2.hotel_address']].apply(lambda x: fuzz.token_set_ratio(str(x['p1.hotel_address']), str(x['p2.hotel_address'])), axis=1)

In [0]:
plt.figure(figsize = (25,14))
plt.subplot(2,2,1)
plt.title('Hotel Address: Using the Ratio' ,color = "g")
sns.distplot(example_remain['address_ratio'], kde=False);
plt.subplot(2,2,2)
plt.title('Hotel Address: Using the Partial Ratio' ,color = "g")
sns.distplot(example_remain['address_partial_ratio'], kde=False);
plt.subplot(2,2,3)
plt.title('Hotel Address: Using the Token Sort Ratio' ,color = "g")
sns.distplot(example_remain['address_token_sort_ratio'], kde=False);
plt.subplot(2,2,4)
plt.title('Hotel Address: Using the Token Set Ratio' ,color = "g")
sns.distplot(example_remain['address_token_set_ratio'], kde=False);

### Hotel City

In [0]:
example_remain['city_ratio'] = example_remain[['p1.city_name', 'p2.city_name']].apply(lambda x: fuzz.ratio(str(x['p1.city_name']), str(x['p2.city_name'])), axis=1)
example_remain['city_partial_ratio'] = example_remain[['p1.city_name', 'p2.city_name']].apply(lambda x: fuzz.partial_ratio(str(x['p1.city_name']), str(x['p2.city_name'])), axis=1)
example_remain['city_token_sort_ratio'] = example_remain[['p1.city_name', 'p2.city_name']].apply(lambda x: fuzz.token_sort_ratio(str(x['p1.city_name']), str(x['p2.city_name'])), axis=1)
example_remain['city_token_set_ratio'] = example_remain[['p1.city_name', 'p2.city_name']].apply(lambda x: fuzz.token_set_ratio(str(x['p1.city_name']), str(x['p2.city_name'])), axis=1)

In [0]:
plt.figure(figsize = (25,14))
plt.subplot(2,2,1)
plt.title('Hotel City: Using the Ratio' ,color = "g")
sns.distplot(example_remain['city_ratio'], kde=False);
plt.subplot(2,2,2)
plt.title('Hotel City: Using the Partial Ratio' ,color = "g")
sns.distplot(example_remain['city_partial_ratio'], kde=False);
plt.subplot(2,2,3)
plt.title('Hotel City: Using the Token Sort Ratio' ,color = "g")
sns.distplot(example_remain['city_token_sort_ratio'], kde=False);
plt.subplot(2,2,4)
plt.title('Hotel City: Using the Token Set Ratio' ,color = "g")
sns.distplot(example_remain['city_token_set_ratio'], kde=False);

## When there is no match

Now we will take unified2_remain dataset and erase all the datapoints that belong the partner2 section of the example_remain dataset. That will leave us with a small dataset that is the partner1 section of example_remain and with a big dataset coming from partner2 that has, by design, no matching pairs.

In [0]:
unified2_remain.head()

In [0]:
bad_unified2_remain = unified2_remain.merge(example_remain, left_on=['key'], right_on=['p2.key'], how='left', indicator=True)
bad_unified2_remain = bad_unified2_remain[bad_unified2_remain['_merge']=='left_only']
bad_unified2_remain = bad_unified2_remain[['key', 'hotel_name', 'city_name', 'country_code', 'hotel_address', 'star_rating', 'postal_code']]
bad_unified2_remain.shape[0]

In [0]:
example1_remain = example_remain[['p1.key', 'p1.hotel_name', 'p1.city_name', 'p1.country_code', 'p1.hotel_address', 'p1.star_rating', 'p1.postal_code']]
example1_remain.shape[0]

Now, for every item on example1_remain we are going to measure the score agianst every element on bad_unified2_remain that is from the same country. Let's define a micro-function for each of the scores and each of the variables (name, city, address) 

In [0]:
def ratio_getNameScores(row, df):  
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['name_score'] = country_df.apply(lambda x: fuzz.ratio(row['p1.hotel_name'], x['hotel_name']), axis=1)
    ret_a_list = country_df['name_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def ratio_getAddressScores(row, df):
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['address_score'] = country_df.apply(lambda x: fuzz.ratio(str(row['p1.hotel_address']), str(x['hotel_address'])), axis=1)
    ret_a_list = country_df['address_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def ratio_getCityScores(row, df):
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['city_score'] = country_df.apply(lambda x: fuzz.ratio(str(row['p1.city_name']), str(x['city_name'])), axis=1)
    ret_a_list = country_df['city_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def partial_getNameScores(row, df):  
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['name_score'] = country_df.apply(lambda x: fuzz.partial_ratio(row['p1.hotel_name'], x['hotel_name']), axis=1)
    ret_a_list = country_df['name_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def partial_getAddressScores(row, df):
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['address_score'] = country_df.apply(lambda x: fuzz.partial_ratio(str(row['p1.hotel_address']), str(x['hotel_address'])), axis=1)
    ret_a_list = country_df['address_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def partial_getCityScores(row, df):
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['city_score'] = country_df.apply(lambda x: fuzz.partial_ratio(str(row['p1.city_name']), str(x['city_name'])), axis=1)
    ret_a_list = country_df['city_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def sort_getNameScores(row, df):  
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['name_score'] = country_df.apply(lambda x: fuzz.token_sort_ratio(row['p1.hotel_name'], x['hotel_name']), axis=1)
    ret_a_list = country_df['name_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def sort_getAddressScores(row, df):
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['address_score'] = country_df.apply(lambda x: fuzz.token_sort_ratio(str(row['p1.hotel_address']), str(x['hotel_address'])), axis=1)
    ret_a_list = country_df['address_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def sort_getCityScores(row, df):
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['city_score'] = country_df.apply(lambda x: fuzz.token_sort_ratio(str(row['p1.city_name']), str(x['city_name'])), axis=1)
    ret_a_list = country_df['city_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def set_getNameScores(row, df):  
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['name_score'] = country_df.apply(lambda x: fuzz.token_set_ratio(row['p1.hotel_name'], x['hotel_name']), axis=1)
    ret_a_list = country_df['name_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def set_getAddressScores(row, df):
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['address_score'] = country_df.apply(lambda x: fuzz.token_set_ratio(str(row['p1.hotel_address']), str(x['hotel_address'])), axis=1)
    ret_a_list = country_df['address_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

def set_getCityScores(row, df):
  country_df = df[df['country_code']==row['p1.country_code']]
  if country_df.shape[0] > 0:
    country_df['city_score'] = country_df.apply(lambda x: fuzz.token_set_ratio(str(row['p1.city_name']), str(x['city_name'])), axis=1)
    ret_a_list = country_df['city_score'].tolist()
  else:
    ret_a_list = [np.nan]
  return ret_a_list

Now let's run each function. This means that for every datapoint on example1_remain (369) we will run all the similarity scores (4) for all the variables (3) for every datapoint on unified2_remain that is of the same country as tehe query. In total we will get about 180.000 similarity measurements of NOT matching hotels. It takes some time to run...

In [0]:
print(1)
example1_remain['ratio_name_score'] = example1_remain.apply(lambda x: ratio_getNameScores(x, bad_unified2_remain), axis=1)
print(2)
example1_remain['ratio_address_score'] = example1_remain.apply(lambda x: ratio_getAddressScores(x, bad_unified2_remain), axis=1)
print(3)
example1_remain['ratio_city_score'] = example1_remain.apply(lambda x: ratio_getCityScores(x, bad_unified2_remain), axis=1)
print(4)
example1_remain['partial_name_score'] = example1_remain.apply(lambda x: partial_getNameScores(x, bad_unified2_remain), axis=1)
print(5)
example1_remain['partial_address_score'] = example1_remain.apply(lambda x: partial_getAddressScores(x, bad_unified2_remain), axis=1)
print(6)
example1_remain['partial_city_score'] = example1_remain.apply(lambda x: partial_getCityScores(x, bad_unified2_remain), axis=1)
print(7)
example1_remain['sort_name_score'] = example1_remain.apply(lambda x: sort_getNameScores(x, bad_unified2_remain), axis=1)
print(8)
example1_remain['sort_address_score'] = example1_remain.apply(lambda x: sort_getAddressScores(x, bad_unified2_remain), axis=1)
print(9)
example1_remain['sort_city_score'] = example1_remain.apply(lambda x: sort_getCityScores(x, bad_unified2_remain), axis=1)
print(10)
example1_remain['set_name_score'] = example1_remain.apply(lambda x: set_getNameScores(x, bad_unified2_remain), axis=1)
print(11)
example1_remain['set_address_score'] = example1_remain.apply(lambda x: set_getAddressScores(x, bad_unified2_remain), axis=1)
print(12)
example1_remain['set_city_score'] = example1_remain.apply(lambda x: set_getCityScores(x, bad_unified2_remain), axis=1)

## Match vs No-Match

We have the scores of the matching datapoints and we also have the score of the not-matching variables. Let's say that for every score type (ratio, partial, sort, set) we sum the scores of the features (name, city, address). Let's say we do a normalized weighted sum. Let's write a function to do that and that returns us some nice graphs that depend on 3 parameters: b, k, p  

**Score = b(name_score) + k(address_score) + p(city_score)**

Let's play with the parameters to get some intuition and great graphs for the presentation

In [0]:
b = 1
k = 1
p = 1

factor = 3/(b+k+p)
b = b*factor
k = k*factor
p = p*factor

from scipy.stats import norm
plt.figure(figsize = (25,14))

# Ratio 
plt.subplot(2,2,1)

scores1a = example1_remain['ratio_name_score']
scores1a = [item for items in scores1a for item in items]
scores1a = [x for x in scores1a if ~np.isnan(x)]
scores1am = [b*i for i in scores1a]

scores2a = example1_remain['ratio_address_score']
scores2a = [item for items in scores2a for item in items]
scores2a = [x for x in scores2a if ~np.isnan(x)]
scores2am = [k*i for i in scores2a]

scores3a = example1_remain['ratio_city_score']
scores3a = [item for items in scores3a for item in items]
scores3a = [x for x in scores3a if ~np.isnan(x)]
scores3am = [p*i for i in scores3a]

score_suma = [x+y+z for x, y, z in zip(scores1am, scores2am, scores3am)]
turth_suma = b*example_remain['name_ratio'] + k*example_remain['address_ratio'] + p*example_remain['city_ratio']

#####
#score_suma = [max(x,y,z) for x,y,z in zip(scores1am, scores2am, scores3am)]
#turth_suma = [max(x,y,z) for x,y,z in zip(b*example_remain['name_ratio'], k*example_remain['address_ratio'], p*example_remain['city_ratio'])]
#####

plt.title('Hotel Name + Hotel Address + Hotel City: Using the Ratio' ,color = "g")
plt.hist(score_suma, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))
plt.hist(turth_suma, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))

# Partial Ratio
plt.subplot(2,2,2)

scores1b = example1_remain['partial_name_score']
scores1b = [item for items in scores1b for item in items]
scores1b = [x for x in scores1b if ~np.isnan(x)]
scores1bm = [b*i for i in scores1b]

scores2b = example1_remain['partial_address_score']
scores2b = [item for items in scores2b for item in items]
scores2b = [x for x in scores2b if ~np.isnan(x)]
scores2bm = [k*i for i in scores2b]

scores3b = example1_remain['partial_city_score']
scores3b = [item for items in scores3b for item in items]
scores3b = [x for x in scores3b if ~np.isnan(x)]
scores3bm = [p*i for i in scores3b]

score_sumb = [x+y+z for x, y, z in zip(scores1bm, scores2bm, scores3bm)]
turth_sumb = b*example_remain['name_partial_ratio'] + k*example_remain['address_partial_ratio'] + p*example_remain['city_partial_ratio']

#####
#score_suma = [max(x,y,z) for x,y,z in zip(scores1bm, scores2bm, scores3bm)]
#turth_suma = [max(x,y,z) for x,y,z in zip(b*example_remain['name_partial_ratio'], k*example_remain['address_partial_ratio'], p*example_remain['city_partial_ratio'])]
#####

plt.title('Hotel Name + Hotel Address + Hotel City: Using the Partial Ratio' ,color = "g")
plt.hist(score_sumb, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))
plt.hist(turth_sumb, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))

# Token Sort Ratio
plt.subplot(2,2,3)

scores1c = example1_remain['sort_name_score']
scores1c = [item for items in scores1c for item in items]
scores1c = [x for x in scores1c if ~np.isnan(x)]
scores1cm = [b*i for i in scores1c]

scores2c = example1_remain['sort_address_score']
scores2c = [item for items in scores2c for item in items]
scores2c = [x for x in scores2c if ~np.isnan(x)]
scores2cm = [k*i for i in scores2c]

scores3c = example1_remain['sort_city_score']
scores3c = [item for items in scores3c for item in items]
scores3c = [x for x in scores3c if ~np.isnan(x)]
scores3cm = [p*i for i in scores3c]

score_sumc = [x+y+z for x, y, z in zip(scores1cm, scores2cm, scores3cm)]
turth_sumc = b*example_remain['name_token_sort_ratio'] + k*example_remain['address_token_sort_ratio'] + p*example_remain['city_token_sort_ratio']

#####
#score_suma = [max(x,y,z) for x,y,z in zip(scores1cm, scores2cm, scores3cm)]
#turth_suma = [max(x,y,z) for x,y,z in zip(b*example_remain['name_token_sort_ratio'], k*example_remain['address_token_sort_ratio'], p*example_remain['city_token_sort_ratio'])]
#####

plt.title('Hotel Name + Hotel Address + Hotel City: Using the Token Sort Ratio' ,color = "g")
plt.hist(score_sumc, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))
plt.hist(turth_sumc, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))

# Token Set Ratio
plt.subplot(2,2,4)

scores1d = example1_remain['set_name_score']
scores1d = [item for items in scores1d for item in items]
scores1d = [x for x in scores1d if ~np.isnan(x)]
scores1dm = [b*i for i in scores1d]

scores2d = example1_remain['set_address_score']
scores2d = [item for items in scores2d for item in items]
scores2d = [x for x in scores2d if ~np.isnan(x)]
scores2dm = [k*i for i in scores2d]

scores3d = example1_remain['set_city_score']
scores3d = [item for items in scores3d for item in items]
scores3d = [x for x in scores3d if ~np.isnan(x)]
scores3dm = [p*i for i in scores3d]

score_sumd = [x+y+z for x, y, z in zip(scores1dm, scores2dm, scores3dm)];
turth_sumd = b*example_remain['name_token_set_ratio'] + k*example_remain['address_token_set_ratio'] + p*example_remain['city_token_set_ratio'];

#####
#score_suma = [max(x,y,z) for x,y,z in zip(scores1dm, scores2dm, scores3dm)]
#turth_suma = [max(x,y,z) for x,y,z in zip(b*example_remain['name_token_set_ratio'], k*example_remain['address_token_set_ratio'], p*example_remain['city_token_set_ratio'])]
#####

plt.title('Hotel Name + Hotel Address + Hotel City: Using the Token Set Ratio' ,color = "g")
plt.hist(score_sumd, normed=True, alpha = 0.5, bins=np.linspace(0,300,30));
plt.hist(turth_sumd, normed=True, alpha = 0.5, bins=np.linspace(0,300,30));

# Filter out by optimal score separation

We found the optimal parameters using optimization of the following objetive function. When using the parameters (b,k,p) = (1,0.35,0.1) under the token_set_ratio we can feel sure to say that if the score is above 280 then it's a match.

In [0]:
def objective(x, score1fake, score2fake, score3fake, score1true, score2true, score3true):
  
  b = x[0]
  k = x[1]
  p = x[2]
  
  factor = 3/(b+k+p)
  b = b*factor
  k = k*factor
  p = p*factor
  
  score1fake = [b*i for i in score1fake]
  score2fake = [k*i for i in score2fake]
  score3fake = [p*i for i in score3fake]

  score1true = [b*i for i in score1true]
  score2true = [k*i for i in score2true]
  score3true = [p*i for i in score3true]
  
  scoreFake = [x+y+z for x, y, z in zip(score1fake, score2fake, score3fake)]
  scoreTrue = [x+y+z for x, y, z in zip(score1true, score2true, score3true)]
  
  maxFake = max(scoreFake)  
  numOfGreater = sum(i > maxFake+10 for i in scoreTrue)
  
  score = numOfGreater
  
  return score

In [0]:
def getScore(row, df):
    
  # Calculate score
  df['scores1d'] = df.apply(lambda x: fuzz.token_set_ratio(row['hotel_name'], x['hotel_name']), axis=1)
  df['scores2d'] = df.apply(lambda x: fuzz.token_set_ratio(str(row['hotel_address']), str(x['hotel_address'])), axis=1)
  df['scores3d'] = df.apply(lambda x: fuzz.token_set_ratio(str(row['city_name']), str(x['city_name'])), axis=1)
  df['tot_score'] = 2.069*df['scores1d'] + 0.72*df['scores2d'] + 0.2*df['scores3d']
  df['match'] = df.apply(lambda x: 1 if x['tot_score']>280 else 0, axis=1)
  
  # Sum the amount of matches
  suma = df['match'].sum()
  
  if suma!=1:
    pred_key = 'bla'
  else:
    pred_key = df[df['match']==1]['key'].values
    pred_key = pred_key[0]

  return pred_key

In [0]:
def runModule(dataframe1, dataframe2, countries=None):

  if countries is None:
    # Get list of available countries on unified dataframe from partner1
    countries = dataframe1['country_code'].unique()

  # Initialize output dataframe
  outDf = pd.DataFrame(columns = ['key', 'hotel_name', 'city_name', 'country_code', 'hotel_address', 'star_rating', 'postal_code', 'predicted_key'])
    
  # Run for each country
  for cou in countries:

    print(cou)

    # Filter dataframes by country    
    df1 = dataframe1[dataframe1['country_code']==cou]
    df2 = dataframe2[dataframe2['country_code']==cou]
    
    if df2.shape[0]<1:
      df1['predicted_key'] = 'bla'
    else:
      # Call the Scoring function to get predicted_key, score and second_score 
      df1['predicted_key'] = df1.apply(lambda x: getScore(x, df2), axis=1)

    # Append to results dataframe
    outDf = outDf.append(df1)

  return outDf

In [0]:
run = 1

if run == 1:
  ttt = runModule(unified1_remain, unified2_remain)
else:
  from google.colab import files
  import pandas as pd
  import io

  uploaded = files.upload()
  ttt = pd.read_csv(io.StringIO(uploaded['A_ttt.csv'].decode('utf-8')))

In [0]:
#SAVEEE
from google.colab import files
caca = ttt.to_csv("ttt.csv", index=False)
files.download('ttt.csv')

In [0]:
ttt.head()

In [0]:
# Let's see how many did we match and how many we didn't
print(ttt[ttt['predicted_key']!='bla'].shape[0])
print(ttt[ttt['predicted_key']=='bla'].shape[0])

In [0]:
# Cut the datapoints that where matched and check for duplicates
matched_by_ratio = ttt[ttt['predicted_key']!='bla']
print(matched_by_ratio[matched_by_ratio.duplicated(subset=['key'], keep=False)].shape[0])
print(matched_by_ratio[matched_by_ratio.duplicated(subset=['predicted_key'], keep=False)].shape[0])

In [0]:
#matched_by_ratio = matched_by_ratio[matched_by_ratio.duplicated(subset=['predicted_key'], keep=False)]
matched_by_ratio_pais = matched_by_ratio[['key', 'predicted_key']]

In [0]:
# Add the key pairs to the matched dataframe
matched = matched.append(matched_by_ratio_pais)
matched.reset_index(drop=True, inplace=True)
matched.shape[0]

In [0]:
print(matched[matched.duplicated(subset=['key'], keep=False)].shape[0])
print(matched[matched.duplicated(subset=['predicted_key'], keep=False)].shape[0])

In [0]:
# Drop the matched hotels from each unified1 and unified2
unified1_remain = unified1.merge(matched, left_on=['key'], right_on=['key'], how='left', indicator=True)
unified1_remain = unified1_remain[unified1_remain['_merge']=='left_only']
unified1_remain.drop(columns=['_merge', 'predicted_key'], inplace=True)
print(unified1_remain.shape[0])

unified2_remain = unified2.merge(matched, left_on=['key'], right_on=['predicted_key'], how='left', indicator=True)
unified2_remain = unified2_remain[unified2_remain['_merge']=='left_only']
unified2_remain.drop(columns=['_merge', 'key_y', 'predicted_key'], inplace=True)
unified2_remain = unified2_remain.rename(columns = {'key_x':'key'})
print(unified2_remain.shape[0])

In [0]:
# Get datapoints inside example that were not yet matched
example_remain = example.merge(matched, left_on=['p1.key'], right_on=['key'], how='left', indicator=True)
example_remain = example_remain[example_remain['_merge']=='left_only']
example_remain.drop(columns=['key', 'predicted_key', '_merge'], inplace=True)
example_remain.shape[0]

In [0]:
example_remain = example_remain.merge(matched, left_on=['p2.key'], right_on=['predicted_key'], how='left', indicator=True)
example_remain = example_remain[example_remain['_merge']=='left_only']
example_remain.drop(columns=['key', 'predicted_key', '_merge'], inplace=True)
example_remain.shape[0]

In [0]:
#SAVEEE
from google.colab import files
caca = matched.to_csv("matched_after_score_th.csv", index=False)
files.download('matched_after_score_th.csv')
caca = unified1_remain.to_csv("unified1_remain_after_score_th.csv", index=False)
files.download('unified1_remain_after_score_th.csv')
caca = unified2_remain.to_csv("unified2_remain_after_score_th.csv", index=False)
files.download('unified2_remain_after_score_th.csv')
caca = example_remain.to_csv("example_remain_after_score_th.csv", index=False)
files.download('example_remain_after_score_th.csv')

In [0]:
# As before, calculate the section of the unified2_remain that has is not labeled
bad_unified2_remain = unified2_remain.merge(example_remain, left_on=['key'], right_on=['p2.key'], how='left', indicator=True)
bad_unified2_remain = bad_unified2_remain[bad_unified2_remain['_merge']=='left_only']
bad_unified2_remain = bad_unified2_remain[['key', 'hotel_name', 'city_name', 'country_code', 'hotel_address', 'star_rating', 'postal_code']]
print(bad_unified2_remain.shape[0])

# Calculate the partner1 section of the labeled data
example1_remain = example_remain[['p1.key', 'p1.hotel_name', 'p1.city_name', 'p1.country_code', 'p1.hotel_address', 'p1.star_rating', 'p1.postal_code']]
print(example1_remain.shape[0])

# A 12 dimensional space

We have a lot of data. We have about 33.250 datapoints that we know for sure are not a match and we also have 137 datapoints that we know are a match. For each datapoint we have 12 features! The score for the name, address and city similarity using 4 different methods. How about we use those features and the big amount of data we have to build a Decision Tree classifier that will be more efficient on splitting the space than our (b, k, p) partition.

In [0]:
# Repeat scoring process for the remaining "labeled" data
example_remain['name_ratio'] = example_remain[['p1.hotel_name', 'p2.hotel_name']].apply(lambda x: fuzz.ratio(x['p1.hotel_name'], x['p2.hotel_name']), axis=1)
example_remain['name_partial_ratio'] = example_remain[['p1.hotel_name', 'p2.hotel_name']].apply(lambda x: fuzz.partial_ratio(x['p1.hotel_name'], x['p2.hotel_name']), axis=1)
example_remain['name_token_sort_ratio'] = example_remain[['p1.hotel_name', 'p2.hotel_name']].apply(lambda x: fuzz.token_sort_ratio(x['p1.hotel_name'], x['p2.hotel_name']), axis=1)
example_remain['name_token_set_ratio'] = example_remain[['p1.hotel_name', 'p2.hotel_name']].apply(lambda x: fuzz.token_set_ratio(x['p1.hotel_name'], x['p2.hotel_name']), axis=1)

example_remain['address_ratio'] = example_remain[['p1.hotel_address', 'p2.hotel_address']].apply(lambda x: fuzz.ratio(str(x['p1.hotel_address']), str(x['p2.hotel_address'])), axis=1)
example_remain['address_partial_ratio'] = example_remain[['p1.hotel_address', 'p2.hotel_address']].apply(lambda x: fuzz.partial_ratio(str(x['p1.hotel_address']), str(x['p2.hotel_address'])), axis=1)
example_remain['address_token_sort_ratio'] = example_remain[['p1.hotel_address', 'p2.hotel_address']].apply(lambda x: fuzz.token_sort_ratio(str(x['p1.hotel_address']), str(x['p2.hotel_address'])), axis=1)
example_remain['address_token_set_ratio'] = example_remain[['p1.hotel_address', 'p2.hotel_address']].apply(lambda x: fuzz.token_set_ratio(str(x['p1.hotel_address']), str(x['p2.hotel_address'])), axis=1)

example_remain['city_ratio'] = example_remain[['p1.city_name', 'p2.city_name']].apply(lambda x: fuzz.ratio(str(x['p1.city_name']), str(x['p2.city_name'])), axis=1)
example_remain['city_partial_ratio'] = example_remain[['p1.city_name', 'p2.city_name']].apply(lambda x: fuzz.partial_ratio(str(x['p1.city_name']), str(x['p2.city_name'])), axis=1)
example_remain['city_token_sort_ratio'] = example_remain[['p1.city_name', 'p2.city_name']].apply(lambda x: fuzz.token_sort_ratio(str(x['p1.city_name']), str(x['p2.city_name'])), axis=1)
example_remain['city_token_set_ratio'] = example_remain[['p1.city_name', 'p2.city_name']].apply(lambda x: fuzz.token_set_ratio(str(x['p1.city_name']), str(x['p2.city_name'])), axis=1)

In [0]:
# Repeat the scoring process for the labeled data from partner1 against the wrong matches of unified2_remain
print(1)
example1_remain['ratio_name_score'] = example1_remain.apply(lambda x: ratio_getNameScores(x, bad_unified2_remain), axis=1)
print(2)
example1_remain['ratio_address_score'] = example1_remain.apply(lambda x: ratio_getAddressScores(x, bad_unified2_remain), axis=1)
print(3)
example1_remain['ratio_city_score'] = example1_remain.apply(lambda x: ratio_getCityScores(x, bad_unified2_remain), axis=1)
print(4)
example1_remain['partial_name_score'] = example1_remain.apply(lambda x: partial_getNameScores(x, bad_unified2_remain), axis=1)
print(5)
example1_remain['partial_address_score'] = example1_remain.apply(lambda x: partial_getAddressScores(x, bad_unified2_remain), axis=1)
print(6)
example1_remain['partial_city_score'] = example1_remain.apply(lambda x: partial_getCityScores(x, bad_unified2_remain), axis=1)
print(7)
example1_remain['sort_name_score'] = example1_remain.apply(lambda x: sort_getNameScores(x, bad_unified2_remain), axis=1)
print(8)
example1_remain['sort_address_score'] = example1_remain.apply(lambda x: sort_getAddressScores(x, bad_unified2_remain), axis=1)
print(9)
example1_remain['sort_city_score'] = example1_remain.apply(lambda x: sort_getCityScores(x, bad_unified2_remain), axis=1)
print(10)
example1_remain['set_name_score'] = example1_remain.apply(lambda x: set_getNameScores(x, bad_unified2_remain), axis=1)
print(11)
example1_remain['set_address_score'] = example1_remain.apply(lambda x: set_getAddressScores(x, bad_unified2_remain), axis=1)
print(12)
example1_remain['set_city_score'] = example1_remain.apply(lambda x: set_getCityScores(x, bad_unified2_remain), axis=1)

In [0]:
# Recalculate the scores that we are going to use for the classifier
b = 1
k = 1
p = 1

factor = 3/(b+k+p)
b = b*factor
k = k*factor
p = p*factor

from scipy.stats import norm
plt.figure(figsize = (25,14))

# Ratio 
plt.subplot(2,2,1)

scores1a = example1_remain['ratio_name_score']
scores1a = [item for items in scores1a for item in items]
scores1a = [x for x in scores1a if ~np.isnan(x)]
scores1am = [b*i for i in scores1a]

scores2a = example1_remain['ratio_address_score']
scores2a = [item for items in scores2a for item in items]
scores2a = [x for x in scores2a if ~np.isnan(x)]
scores2am = [k*i for i in scores2a]

scores3a = example1_remain['ratio_city_score']
scores3a = [item for items in scores3a for item in items]
scores3a = [x for x in scores3a if ~np.isnan(x)]
scores3am = [p*i for i in scores3a]

score_suma = [x+y+z for x, y, z in zip(scores1am, scores2am, scores3am)]
turth_suma = b*example_remain['name_ratio'] + k*example_remain['address_ratio'] + p*example_remain['city_ratio']

#####
#score_suma = [max(x,y,z) for x,y,z in zip(scores1am, scores2am, scores3am)]
#turth_suma = [max(x,y,z) for x,y,z in zip(b*example_remain['name_ratio'], k*example_remain['address_ratio'], p*example_remain['city_ratio'])]
#####

plt.title('Hotel Name + Hotel Address + Hotel City: Using the Ratio' ,color = "g")
plt.hist(score_suma, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))
plt.hist(turth_suma, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))

# Partial Ratio
plt.subplot(2,2,2)

scores1b = example1_remain['partial_name_score']
scores1b = [item for items in scores1b for item in items]
scores1b = [x for x in scores1b if ~np.isnan(x)]
scores1bm = [b*i for i in scores1b]

scores2b = example1_remain['partial_address_score']
scores2b = [item for items in scores2b for item in items]
scores2b = [x for x in scores2b if ~np.isnan(x)]
scores2bm = [k*i for i in scores2b]

scores3b = example1_remain['partial_city_score']
scores3b = [item for items in scores3b for item in items]
scores3b = [x for x in scores3b if ~np.isnan(x)]
scores3bm = [p*i for i in scores3b]

score_sumb = [x+y+z for x, y, z in zip(scores1bm, scores2bm, scores3bm)]
turth_sumb = b*example_remain['name_partial_ratio'] + k*example_remain['address_partial_ratio'] + p*example_remain['city_partial_ratio']

#####
#score_suma = [max(x,y,z) for x,y,z in zip(scores1bm, scores2bm, scores3bm)]
#turth_suma = [max(x,y,z) for x,y,z in zip(b*example_remain['name_partial_ratio'], k*example_remain['address_partial_ratio'], p*example_remain['city_partial_ratio'])]
#####

plt.title('Hotel Name + Hotel Address + Hotel City: Using the Partial Ratio' ,color = "g")
plt.hist(score_sumb, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))
plt.hist(turth_sumb, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))

# Token Sort Ratio
plt.subplot(2,2,3)

scores1c = example1_remain['sort_name_score']
scores1c = [item for items in scores1c for item in items]
scores1c = [x for x in scores1c if ~np.isnan(x)]
scores1cm = [b*i for i in scores1c]

scores2c = example1_remain['sort_address_score']
scores2c = [item for items in scores2c for item in items]
scores2c = [x for x in scores2c if ~np.isnan(x)]
scores2cm = [k*i for i in scores2c]

scores3c = example1_remain['sort_city_score']
scores3c = [item for items in scores3c for item in items]
scores3c = [x for x in scores3c if ~np.isnan(x)]
scores3cm = [p*i for i in scores3c]

score_sumc = [x+y+z for x, y, z in zip(scores1cm, scores2cm, scores3cm)]
turth_sumc = b*example_remain['name_token_sort_ratio'] + k*example_remain['address_token_sort_ratio'] + p*example_remain['city_token_sort_ratio']

#####
#score_suma = [max(x,y,z) for x,y,z in zip(scores1cm, scores2cm, scores3cm)]
#turth_suma = [max(x,y,z) for x,y,z in zip(b*example_remain['name_token_sort_ratio'], k*example_remain['address_token_sort_ratio'], p*example_remain['city_token_sort_ratio'])]
#####

plt.title('Hotel Name + Hotel Address + Hotel City: Using the Token Sort Ratio' ,color = "g")
plt.hist(score_sumc, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))
plt.hist(turth_sumc, normed=True, alpha = 0.5, bins=np.linspace(0,300,30))

# Token Set Ratio
plt.subplot(2,2,4)

scores1d = example1_remain['set_name_score']
scores1d = [item for items in scores1d for item in items]
scores1d = [x for x in scores1d if ~np.isnan(x)]
scores1dm = [b*i for i in scores1d]

scores2d = example1_remain['set_address_score']
scores2d = [item for items in scores2d for item in items]
scores2d = [x for x in scores2d if ~np.isnan(x)]
scores2dm = [k*i for i in scores2d]

scores3d = example1_remain['set_city_score']
scores3d = [item for items in scores3d for item in items]
scores3d = [x for x in scores3d if ~np.isnan(x)]
scores3dm = [p*i for i in scores3d]

score_sumd = [x+y+z for x, y, z in zip(scores1dm, scores2dm, scores3dm)];
turth_sumd = b*example_remain['name_token_set_ratio'] + k*example_remain['address_token_set_ratio'] + p*example_remain['city_token_set_ratio'];

#####
#score_suma = [max(x,y,z) for x,y,z in zip(scores1dm, scores2dm, scores3dm)]
#turth_suma = [max(x,y,z) for x,y,z in zip(b*example_remain['name_token_set_ratio'], k*example_remain['address_token_set_ratio'], p*example_remain['city_token_set_ratio'])]
#####

plt.title('Hotel Name + Hotel Address + Hotel City: Using the Token Set Ratio' ,color = "g")
plt.hist(score_sumd, normed=True, alpha = 0.5, bins=np.linspace(0,300,30));
plt.hist(turth_sumd, normed=True, alpha = 0.5, bins=np.linspace(0,300,30));

In [0]:
# Build the NO-match dataframe
d_fake = {'scores1a': scores1a, 'scores2a': scores2a, 'scores3a': scores3a, 
          'scores1b': scores1b, 'scores2b': scores2b, 'scores3b': scores3b, 
          'scores1c': scores1c, 'scores2c': scores2c, 'scores3c': scores3c, 
          'scores1d': scores1d, 'scores2d': scores2d, 'scores3d': scores3d}
df_fake = pd.DataFrame(data=d_fake)
df_fake['label']=0

# Build the YES-match dataframe
d_true = {'scores1a': example_remain['name_ratio'], 'scores2a': example_remain['address_ratio'],
          'scores3a': example_remain['city_ratio'], 'scores1b': example_remain['name_partial_ratio'],
          'scores2b': example_remain['address_partial_ratio'], 'scores3b': example_remain['city_partial_ratio'],
          'scores1c': example_remain['name_token_sort_ratio'], 'scores2c': example_remain['address_token_sort_ratio'],
          'scores3c': example_remain['city_token_sort_ratio'], 'scores1d': example_remain['name_token_set_ratio'],
          'scores2d': example_remain['address_token_set_ratio'], 'scores3d': example_remain['city_token_set_ratio']}
df_true = pd.DataFrame(data=d_true)
df_true['label']=1

In [0]:
# Merge the dataframes, reset_index and take a look
df = df_true.append(df_fake)
df = df.sample(frac=1).reset_index(drop=True)
df.shape[0]

In [0]:
# SAVEEEEEE
from google.colab import files
caca = df.to_csv("A_12dim.csv", index=False)
files.download('A_12dim.csv')

In [0]:
features = ['scores1a', 'scores1b', 'scores1c', 'scores1d', 'scores2a', 'scores2b', 'scores2c', 'scores2d', 'scores3a', 'scores3b', 'scores3c', 'scores3d', 'label']
#features = ['scores1a', 'scores1c', 'scores1d', 'scores2c', 'scores2d', 'label']
#features = ['scores1a', 'scores1d', 'scores2c', 'scores2d', 'scores3b', 'label']
#features = ['scores3b', 'scores3d', 'label']

Name Ratio  
Name Set  
Name Sort  
Address Set  
Address Sort  

## XGBoost

In [0]:
# Train - Test split
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df[features], test_size=0.3)
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

# Features and Label split
X_train = df_train.drop(columns=['label'])
Y_train = df_train['label']
X_test = df_test.drop(columns=['label'])
Y_test = df_test['label']

from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=400, objective = 'binary:logistic', scale_pos_weight=1)
model = model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
Y_proba = model.predict_proba(X_test)

from sklearn.metrics import confusion_matrix
conf = confusion_matrix(Y_test, Y_pred)
print(conf)

a = conf[0][0]
b = conf[0][1]
c = conf[1][0]
d = conf[1][1]

acc = 1- b*c/((c+d)*(c+d))
cov = d/(c+d) - (b/(c+d))*(d/(c+d)) + (b/(c+d))*(c/(c+d))

print(acc)
print(cov)

## Logistic Regression

In [0]:
# Train - Test split
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df[features], test_size=0.02)
df_train.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

# Features and Label split
X_train = df_train.drop(columns=['label'])
Y_train = df_train['label']
X_test = df_test.drop(columns=['label'])
Y_test = df_test['label']

from sklearn import linear_model
from sklearn import ensemble
model = linear_model.LogisticRegression(class_weight = {0:1,1:5}).fit(X_train, Y_train)

Y_pred = model.predict(X_test)
Y_prob = model.predict_proba(X_test)

from sklearn.metrics import confusion_matrix
conf = confusion_matrix(Y_test, Y_pred)
print(conf)

a = conf[0][0]
b = conf[0][1]
c = conf[1][0]
d = conf[1][1]

acc = 1- b*c/((c+d)*(c+d))
cov = d/(c+d) - (b/(c+d))*(d/(c+d)) + (b/(c+d))*(c/(c+d))

print(acc)
print(cov)

## Applying the model

We have unified1_remain and unified2_remain, each with 2900 datapoints that include 137 datapoints that belong to the examples dataset.

### Making a matching key function 

What we should do now is make a function that for any given datapoint from unified1_remain it calculates the 12 different scores that we have seen for every single point on unified2_remain that is from the same country as the query point. Then those 12 scores should go trough the logostic regression model and:
- In case there are no matches the datapoint has no match
- In case there is a single match the datapoint has a match
- In case there are more than 2 matches the datapoint has NO match



In [0]:
def getMatchingKey(row, dfi, features):
  
  df = dfi[dfi['country_code']==row['country_code']]
  
  # Calculate score for all 12 features 
  df['scores1a'] = df.apply(lambda x: fuzz.ratio(x['hotel_name'], row['hotel_name']), axis=1)
  df['scores2a'] = df.apply(lambda x: fuzz.ratio(str(x['hotel_address']), str(row['hotel_address'])), axis=1)
  df['scores3a'] = df.apply(lambda x: fuzz.ratio(str(x['city_name']), str(row['city_name'])), axis=1)
  df['scores1b'] = df.apply(lambda x: fuzz.partial_ratio(x['hotel_name'], row['hotel_name']), axis=1)
  df['scores2b'] = df.apply(lambda x: fuzz.partial_ratio(str(x['hotel_address']), str(row['hotel_address'])), axis=1)
  df['scores3b'] = df.apply(lambda x: fuzz.partial_ratio(str(x['city_name']), str(row['city_name'])), axis=1)
  df['scores1c'] = df.apply(lambda x: fuzz.token_sort_ratio(x['hotel_name'], row['hotel_name']), axis=1)
  df['scores2c'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['hotel_address']), str(row['hotel_address'])), axis=1)
  df['scores3c'] = df.apply(lambda x: fuzz.token_sort_ratio(str(x['city_name']), str(row['city_name'])), axis=1)
  df['scores1d'] = df.apply(lambda x: fuzz.token_set_ratio(x['hotel_name'], row['hotel_name']), axis=1)
  df['scores2d'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['hotel_address']), str(row['hotel_address'])), axis=1)
  df['scores3d'] = df.apply(lambda x: fuzz.token_set_ratio(str(x['city_name']), str(row['city_name'])), axis=1)
  
  # Call Model
  suma = model.predict(df[features]).sum()
  if suma != 1:
    pred_key = 'bla'
  else:  
    df['prediction'] = model.predict(df[features])
    pred_key = df[df['prediction']==1]['key'].values
    pred_key = pred_key[0]
  
  return pred_key

In [0]:
caca1 = unified1_remain.copy()
caca2 = unified2_remain.copy()
caca1 = caca1
features = ['scores1a', 'scores1b', 'scores1c', 'scores1d', 'scores2a', 'scores2b', 'scores2c', 'scores2d', 'scores3a', 'scores3b', 'scores3c', 'scores3d']

In [0]:
caca1['predicted_key'] = caca1.apply(lambda x: getMatchingKey(x, caca2, features), axis=1)

In [0]:
# SAVEEEEEE
from google.colab import files
hrgtf = caca1.to_csv("cacaaa.csv", index=False)
files.download('cacaaa.csv')

In [0]:
# LOADDDD
  from google.colab import files
  import pandas as pd
  import io

  uploaded = files.upload()
  matched = pd.read_csv(io.StringIO(uploaded['matched_after_score_th.csv'].decode('utf-8')))

In [0]:
# LOADDDD
  from google.colab import files
  import pandas as pd
  import io

  uploaded = files.upload()
  caca1 = pd.read_csv(io.StringIO(uploaded['cacaaa.csv'].decode('utf-8')))

In [0]:
# Let's see how many did we match and how many we didn't
print(caca1[caca1['predicted_key']!='bla'].shape[0])
print(caca1[caca1['predicted_key']=='bla'].shape[0])

In [0]:
# Cut the datapoints that where matched and check for duplicates
matched_by_logistic = caca1[caca1['predicted_key']!='bla']
print(matched_by_logistic[matched_by_logistic.duplicated(subset=['key'], keep=False)].shape[0])
print(matched_by_logistic[matched_by_logistic.duplicated(subset=['predicted_key'], keep=False)].shape[0])

In [0]:
# Delete duplicates
matched_by_logistic = matched_by_logistic.drop_duplicates(subset='predicted_key', keep=False)
print(matched_by_logistic[matched_by_logistic.duplicated(subset=['key'], keep=False)].shape[0])
print(matched_by_logistic[matched_by_logistic.duplicated(subset=['predicted_key'], keep=False)].shape[0])

In [0]:
#matched_by_ratio = matched_by_ratio[matched_by_ratio.duplicated(subset=['predicted_key'], keep=False)]
matched_by_logistic_pairs = matched_by_logistic[['key', 'predicted_key']]

In [0]:
# Add the key pairs to the matched dataframe
matched = matched.append(matched_by_logistic_pairs)
matched.reset_index(drop=True, inplace=True)
matched.shape[0]

In [0]:
print(matched[matched.duplicated(subset=['key'], keep=False)].shape[0])
print(matched[matched.duplicated(subset=['predicted_key'], keep=False)].shape[0])

In [0]:
# Drop the matched hotels from each unified1 and unified2
unified1_remain = unified1.merge(matched, left_on=['key'], right_on=['key'], how='left', indicator=True)
unified1_remain = unified1_remain[unified1_remain['_merge']=='left_only']
unified1_remain.drop(columns=['_merge', 'predicted_key'], inplace=True)
print(unified1_remain.shape[0])

unified2_remain = unified2.merge(matched, left_on=['key'], right_on=['predicted_key'], how='left', indicator=True)
unified2_remain = unified2_remain[unified2_remain['_merge']=='left_only']
unified2_remain.drop(columns=['_merge', 'key_y', 'predicted_key'], inplace=True)
unified2_remain = unified2_remain.rename(columns = {'key_x':'key'})
print(unified2_remain.shape[0])

In [0]:
# Get datapoints inside example that were not yet matched
example_remain = example.merge(matched, left_on=['p1.key'], right_on=['key'], how='left', indicator=True)
example_remain = example_remain[example_remain['_merge']=='left_only']
example_remain.drop(columns=['key', 'predicted_key', '_merge'], inplace=True)
example_remain.shape[0]

In [0]:
example_remain = example_remain.merge(matched, left_on=['p2.key'], right_on=['predicted_key'], how='left', indicator=True)
example_remain = example_remain[example_remain['_merge']=='left_only']
example_remain.drop(columns=['key', 'predicted_key', '_merge'], inplace=True)
example_remain.shape[0]

In [0]:
#SAVEEE
from google.colab import files
caca = matched.to_csv("matched_after_logistic.csv", index=False)
files.download('matched_after_logistic.csv')
caca = unified1_remain.to_csv("unified1_remain_after_logistic.csv", index=False)
files.download('unified1_remain_after_logistic.csv')
caca = unified2_remain.to_csv("unified2_remain_after_logistic.csv", index=False)
files.download('unified2_remain_after_logistic.csv')
caca = example_remain.to_csv("example_remain_after_logistic.csv", index=False)
files.download('example_remain_after_logistic.csv')

In [0]:
# As before, calculate the section of the unified2_remain that has is not labeled
bad_unified2_remain = unified2_remain.merge(example_remain, left_on=['key'], right_on=['p2.key'], how='left', indicator=True)
bad_unified2_remain = bad_unified2_remain[bad_unified2_remain['_merge']=='left_only']
bad_unified2_remain = bad_unified2_remain[['key', 'hotel_name', 'city_name', 'country_code', 'hotel_address', 'star_rating', 'postal_code']]
print(bad_unified2_remain.shape[0])

# Calculate the partner1 section of the labeled data
example1_remain = example_remain[['p1.key', 'p1.hotel_name', 'p1.city_name', 'p1.country_code', 'p1.hotel_address', 'p1.star_rating', 'p1.postal_code']]
print(example1_remain.shape[0])

# Final Retouch

If it's the exact same name, it's a match

In [0]:
temp1 = unified1_remain.copy()
temp2 = unified2_remain.copy()
temp1['hotel_name2'] = temp1['hotel_name'].str.lower()
temp1['city_name2'] = temp1['city_name'].str.lower()
temp1['country_code2'] = temp1['country_code'].str.lower()
temp1['hotel_address2'] = temp1['hotel_address'].str.lower()
temp2['hotel_name2'] = temp2['hotel_name'].str.lower()
temp2['city_name2'] = temp2['city_name'].str.lower()
temp2['country_code2'] = temp2['country_code'].str.lower()
temp2['hotel_address2'] = temp2['hotel_address'].str.lower()

In [0]:
same_name = pd.merge(temp1, temp2, how='inner', on=['hotel_name2', 'country_code2'])
same_name = same_name[['key_x', 'key_y']]
same_name.columns = ['key', 'predicted_key']
same_name.shape[0]

In [0]:
matched = matched.append(same_name)
matched = matched.reset_index(drop=True)
matched.shape[0]

In [0]:
same_address = pd.merge(temp1, temp2, how='inner', on=['hotel_address2', 'country_code2'])
same_address = same_address[['key_x', 'key_y']]
same_address.columns = ['key', 'predicted_key']
same_address.shape[0]

In [0]:
matched = matched.append(same_address)
matched = matched.reset_index(drop=True)
matched.shape[0]

In [0]:
print(matched[matched.duplicated(subset=['key'], keep=False)].shape[0])
print(matched[matched.duplicated(subset=['predicted_key'], keep=False)].shape[0])

In [0]:
# Drop the matched hotels from each unified1 and unified2
unified1_remain = unified1.merge(matched, left_on=['key'], right_on=['key'], how='left', indicator=True)
unified1_remain = unified1_remain[unified1_remain['_merge']=='left_only']
unified1_remain.drop(columns=['_merge', 'predicted_key'], inplace=True)
print(unified1_remain.shape[0])

unified2_remain = unified2.merge(matched, left_on=['key'], right_on=['predicted_key'], how='left', indicator=True)
unified2_remain = unified2_remain[unified2_remain['_merge']=='left_only']
unified2_remain.drop(columns=['_merge', 'key_y', 'predicted_key'], inplace=True)
unified2_remain = unified2_remain.rename(columns = {'key_x':'key'})
print(unified2_remain.shape[0])

In [0]:
# Get datapoints inside example that were not yet matched
example_remain = example.merge(matched, left_on=['p1.key'], right_on=['key'], how='left', indicator=True)
example_remain = example_remain[example_remain['_merge']=='left_only']
example_remain.drop(columns=['key', 'predicted_key', '_merge'], inplace=True)
example_remain.shape[0]

In [0]:
example_remain = example_remain.merge(matched, left_on=['p2.key'], right_on=['predicted_key'], how='left', indicator=True)
example_remain = example_remain[example_remain['_merge']=='left_only']
example_remain.drop(columns=['key', 'predicted_key', '_merge'], inplace=True)
example_remain.shape[0]

In [0]:
# As before, calculate the section of the unified2_remain that has is not labeled
bad_unified2_remain = unified2_remain.merge(example_remain, left_on=['key'], right_on=['p2.key'], how='left', indicator=True)
bad_unified2_remain = bad_unified2_remain[bad_unified2_remain['_merge']=='left_only']
bad_unified2_remain = bad_unified2_remain[['key', 'hotel_name', 'city_name', 'country_code', 'hotel_address', 'star_rating', 'postal_code']]
print(bad_unified2_remain.shape[0])

# Calculate the partner1 section of the labeled data
example1_remain = example_remain[['p1.key', 'p1.hotel_name', 'p1.city_name', 'p1.country_code', 'p1.hotel_address', 'p1.star_rating', 'p1.postal_code']]
print(example1_remain.shape[0])

# Load and Save

In [0]:
# LOADDDD
  from google.colab import files
  import pandas as pd
  import io

  uploaded = files.upload()
  matched = pd.read_csv(io.StringIO(uploaded['matched_final.csv'].decode('utf-8')))

In [0]:
matched.head()

In [0]:
# Drop the matched hotels from each unified1 and unified2
unified1_remain = unified1.merge(matched, left_on=['key'], right_on=['key'], how='left', indicator=True)
unified1_remain = unified1_remain[unified1_remain['_merge']=='left_only']
unified1_remain.drop(columns=['_merge', 'predicted_key'], inplace=True)
print(unified1_remain.shape[0])

unified2_remain = unified2.merge(matched, left_on=['key'], right_on=['predicted_key'], how='left', indicator=True)
unified2_remain = unified2_remain[unified2_remain['_merge']=='left_only']
unified2_remain.drop(columns=['_merge', 'key_y', 'predicted_key'], inplace=True)
unified2_remain = unified2_remain.rename(columns = {'key_x':'key'})
print(unified2_remain.shape[0])

In [0]:
# Get datapoints inside example that were not yet matched
example_remain = example.merge(matched, left_on=['p1.key'], right_on=['key'], how='left', indicator=True)
example_remain = example_remain[example_remain['_merge']=='left_only']
example_remain.drop(columns=['key', 'predicted_key', '_merge'], inplace=True)
example_remain.shape[0]

In [0]:
example_remain = example_remain.merge(matched, left_on=['p2.key'], right_on=['predicted_key'], how='left', indicator=True)
example_remain = example_remain[example_remain['_merge']=='left_only']
example_remain.drop(columns=['key', 'predicted_key', '_merge'], inplace=True)
example_remain.shape[0]

In [0]:
# As before, calculate the section of the unified2_remain that has is not labeled
bad_unified2_remain = unified2_remain.merge(example_remain, left_on=['key'], right_on=['p2.key'], how='left', indicator=True)
bad_unified2_remain = bad_unified2_remain[bad_unified2_remain['_merge']=='left_only']
bad_unified2_remain = bad_unified2_remain[['key', 'hotel_name', 'city_name', 'country_code', 'hotel_address', 'star_rating', 'postal_code']]
print(bad_unified2_remain.shape[0])

# Calculate the partner1 section of the labeled data
example1_remain = example_remain[['p1.key', 'p1.hotel_name', 'p1.city_name', 'p1.country_code', 'p1.hotel_address', 'p1.star_rating', 'p1.postal_code']]
print(example1_remain.shape[0])

In [0]:
matched.columns = [['p1.key', 'p2.key']]
matched.head()

In [0]:
# Import example data to delete those points
import pandas as pd
downloaded = drive.CreateFile({'id':"1JaZukEjWt_EqDVjvcmFOmdz__opIMOlw"})   # replace the id with id of file you want to access
downloaded.GetContentFile('example.xlsx') 
example = pd.read_excel('example.xlsx')

In [0]:
example = example[['p1.key', 'p2.key']]
example.columns = [['p1.key', 'p2.key']]
example.head()

In [0]:
partner1[partner1['key']=='BD56C41BC2C68FB076F76362DEF0750D'].head()

In [0]:
partner2[partner2['key']=='BD56C41BC2C68FB076F76362DEF0750D'].head()

Give it a try

In [0]:
matched_sample = matched.sample(n=1)
key1 = matched_sample['P1.key'].values[0]
key2 = matched_sample['P2.key'].values[0]
partner1[partner1['key']==key1].head()

In [0]:
partner2[partner2['key']==key2].head()

In [0]:
# SAVEEEEEE
from google.colab import files
caca = matched.to_csv("matched.csv", index=False)
files.download('matched.csv')